# Cross-Layer Equalization and Bias Correction

This notebook contains an example of how to use AIMET to apply Cross-Layer Equalization (CLE) and Bias Correction (BC). CLE and BC are post-training quantization techniques for improving the quantized accuracy of a model. These techniques help recover quantized accuracy when the model quantization is sensitive to parameter quantization as opposed to activation quantization.

CLE does not need any data samples. BC may optionally need unlabeled data samples. 

## Techniques

**Cross-layer equalization**

AIMET performs the following steps when running CLE:

1. Batch norm (BN) Folding: Folds BN layers into convolution (Conv) layers immediate before or after the Conv layers.
2. Cross-layer scaling: For a set of consecutive Conv layers, equalizes the range of tensor values per-channel by scaling their weight tensor values.
3. High bias folding: Cross-layer scaling may result in high bias parameter values for some layers. This technique folds some of a layer's

**Bias Correction**  

Quantization sometimes leads to a shift in layer outputs. Bias correction helps correct this shift by adjusting the bias parameters of that layer. This step is optional, and is applied after CLE.

## Overall flow

This example performs the following steps:

1. Instantiate the example evaluation and training pipeline
2. Load the FP32 model and evaluate the model to find the baseline FP32 accuracy
3. Create a quantization simulation model (with fake quantization ops inserted) and evaluate the quantized simuation model
4. Apply CLE, BC, and high-bias folding and evaluate the simulation model

<div class="alert alert-info">

Note

This notebook does not show state-of-the-art results. For example, it uses a relatively quantization-friendly model (Resnet18). Also, some optimization parameters like number of fine-tuning epochs are chosen to improve execution speed in the notebook.

</div>

---

## Dataset

This example does image classification on the ImageNet dataset. If you already have a version of the data set, use that. Otherwise download the data set, for example from https://image-net.org/challenges/LSVRC/2012/index .

<div class="alert alert-info">

Note

The dataloader provided in this example relies on these features of the ImageNet data set:

- Subfolders `train` for the training samples and `val` for the validation samples. See the [pytorch dataset description](https://pytorch.org/vision/0.8/_modules/torchvision/datasets/imagenet.html) for more details.
- One subdirectory per class, and one file per image sample.

</div>

<div class="alert alert-info">

Note

To speed up the execution of this notebook, you can use a reduced subset of the ImageNet dataset. For example: The entire ILSVRC2012 dataset has 1000 classes, 1000 training samples per class and 50 validation samples per class. However, for the purpose of running this notebook, you can reduce the dataset to, say, two samples per class.

</div>

Edit the cell below to specify the directory where the downloaded ImageNet dataset is saved.

In [1]:
DATASET_DIR = '/home/shayan/Desktop/aimet/Examples/torch/quantization/'         # Replace this path with a real directory

---
## 1. Instantiate the example training and validation pipeline

**Use the following training and validation loop for the image classification task.**

Things to note:

- AIMET does not put limitations on how the training and validation pipeline is written. AIMET modifies the user's model to create a QuantizationSim model, which is still a PyTorch model. The QuantizationSim model can be used in place of the original model when doing inference or training.
- AIMET doesn not put limitations on the interface of the `evaluate()` or `train()` methods. You should be able to use your existing evaluate and train routines as-is.


In [32]:
# ! pip install git+https://github.com/modestyachts/ImageNetV2_pytorch


# from imagenetv2_pytorch import ImageNetV2Dataset


# images = ImageNetV2Dataset()

In [3]:
import sys
sys.path.append("/home/shayan/Desktop/aimet/")

import os
import torch
from Examples.common import image_net_config
from Examples.torch.utils.image_net_evaluator import ImageNetEvaluator
from Examples.torch.utils.image_net_trainer import ImageNetTrainer
from Examples.torch.utils.image_net_data_loader import ImageNetDataLoader

sys.path.remove("/home/shayan/Desktop/aimet/")

class ImageNetDataPipeline:

    @staticmethod
    def get_val_dataloader() -> torch.utils.data.DataLoader:
        """
        Instantiates a validation dataloader for ImageNet dataset and returns it
        """
        data_loader = ImageNetDataLoader(DATASET_DIR,
                                         image_size=image_net_config.dataset['image_size'],
                                         batch_size=image_net_config.evaluation['batch_size'],
                                         is_training=False,
                                         num_workers=image_net_config.evaluation['num_workers']).data_loader
        return data_loader

    @staticmethod
    def evaluate(model: torch.nn.Module, use_cuda: bool) -> float:
        """
        Given a torch model, evaluates its Top-1 accuracy on the dataset
        :param model: the model to evaluate
        :param iterations: the number of batches to be used to evaluate the model. A value of 'None' means the model will be
                           evaluated on the entire dataset once.
        :param use_cuda: whether or not the GPU should be used.
        """
        evaluator = ImageNetEvaluator(DATASET_DIR, image_size=image_net_config.dataset['image_size'],
                                      batch_size=image_net_config.evaluation['batch_size'],
                                      num_workers=image_net_config.evaluation['num_workers'])

        return evaluator.evaluate(model, iterations=None, use_cuda=use_cuda)

---

## 2. Load the model and evaluate to get a baseline FP32 accuracy score

**2.1 Load a pretrained resnet18 model from torchvision.** 

You can load any pretrained PyTorch model instead.

In [4]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:02<00:00, 20.3MB/s]


AIMET quantization simulation requires the model definition to follow certain guidelines. For example, functionals defined in the forward pass should be changed to the equivalent **torch.nn.Module**.
The [AIMET user guide](https://quic.github.io/aimet-pages/releases/latest/user_guide/index.html) lists all these guidelines.

**2.2 Use the following ModelPreparer API call to automate the model definition changes required to comply with the AIMET guidelines.** 

The call uses the graph transformation feature available in PyTorch 1.9+.

In [5]:
from aimet_torch.model_preparer import prepare_model

model = prepare_model(model)

2024-12-20 10:41:42,890 - root - INFO - AIMET
2024-12-20 10:41:42,953 - ModelPreparer - INFO - Functional         : Adding new module for node: {layer1.0.module_add} 
2024-12-20 10:41:42,954 - ModelPreparer - INFO - Reused/Duplicate   : Adding new module for node: {layer1.0.module_relu_1} 
2024-12-20 10:41:42,955 - ModelPreparer - INFO - Functional         : Adding new module for node: {layer1.1.module_add_1} 
2024-12-20 10:41:42,955 - ModelPreparer - INFO - Reused/Duplicate   : Adding new module for node: {layer1.1.module_relu_1} 
2024-12-20 10:41:42,956 - ModelPreparer - INFO - Functional         : Adding new module for node: {layer2.0.module_add_2} 
2024-12-20 10:41:42,956 - ModelPreparer - INFO - Reused/Duplicate   : Adding new module for node: {layer2.0.module_relu_1} 
2024-12-20 10:41:42,956 - ModelPreparer - INFO - Functional         : Adding new module for node: {layer2.1.module_add_3} 
2024-12-20 10:41:42,957 - ModelPreparer - INFO - Reused/Duplicate   : Adding new module for 

---

**2.3 Decide whether to place the model on a CPU or CUDA device.** 

This example uses CUDA if it is available. You can change this logic and force a device placement if needed.

In [6]:
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    model.to(torch.device('cuda'))
use_cuda

False

In [8]:
dataloader = ImageNetDataPipeline.get_val_dataloader()


2024-12-20 10:42:57,506 - Dataloader - INFO - Dataset consists of 3 images in 1 classes


In [15]:
def forward(model, dataloader):
    model.eval()
    with torch.no_grad():
        for path, batch, _ in dataloader:
            print(path)
            output = model(batch)
            break
    return output

fp32_output = forward(model, dataloader)
fp32_output.shape

('/home/shayan/Desktop/aimet/Examples/torch/quantization/val/9/cf135f199d8c7a9d0dce9aa35acfb4c70c14e0aa.jpeg',)


torch.Size([1, 1000])

---

**2.4 Compute the floating point 32-bit (FP32) accuracy of this model using the evaluate() routine.**

In [7]:
accuracy = ImageNetDataPipeline.evaluate(model, use_cuda)
print(accuracy)

2024-12-17 07:59:23,709 - Dataloader - INFO - Dataset consists of 3 images in 1 classes
2024-12-17 07:59:23,710 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-17 07:59:23,712 - Eval - INFO - Evaluating nn.Module for 3 iterations with batch_size 1


 67%|██████▋   | 2/3 [00:00<00:00, 12.35it/s]

tensor([[-3.2675e+00, -2.7024e+00, -2.2966e+00, -2.4265e+00, -5.4076e-01,
          2.0593e+00,  9.8437e-01,  2.8421e-01,  2.5248e+00,  1.0226e+01,
         -7.4343e-01,  9.3748e-01, -8.5360e-01, -1.7097e+00, -4.3297e+00,
         -2.3667e+00, -1.3821e+00, -4.5345e+00, -2.3896e+00, -2.4849e+00,
         -4.5818e+00, -1.4923e+00, -1.0502e+00,  1.9651e+00, -8.0901e-01,
         -2.8727e+00, -7.8400e-01, -1.8091e+00, -2.0265e+00, -2.6331e-01,
         -1.1333e+00, -9.2206e-01,  5.5013e-01, -9.7816e-01, -3.1677e-01,
         -4.0287e+00, -1.3225e+00, -4.2696e+00,  7.1298e-01, -7.3434e-01,
         -2.6860e+00, -2.9846e+00, -3.1025e+00,  1.4333e+00, -1.7502e+00,
          7.9281e-01, -3.0649e+00, -1.2510e+00, -6.4412e-02,  2.2988e+00,
          2.0278e+00,  9.0849e-01, -2.5991e+00, -3.1488e+00, -1.6250e+00,
         -4.0862e+00, -2.5259e+00, -2.9284e+00, -3.3110e+00, -2.8427e+00,
          4.5466e-01,  5.4950e-01, -1.6668e+00,  3.2066e+00, -2.4826e+00,
         -6.8117e-01,  4.7195e-01,  2.

100%|██████████| 3/3 [00:00<00:00, 15.76it/s]

tensor([[-3.7602e+00, -5.1006e+00, -1.5845e+00, -1.3637e+00,  7.1186e-01,
          1.7239e+00,  2.2597e+00,  5.1177e+00,  5.0591e+00,  2.2972e+01,
          9.2571e-02, -1.1743e+00,  3.0293e-01, -1.3205e+00, -1.7226e+00,
          1.6729e-01, -1.1525e+00, -9.4002e-01,  3.0077e+00, -2.2720e+00,
          9.4055e-01,  4.2800e+00,  6.8286e+00,  8.9723e+00,  3.6879e+00,
         -6.6410e-01,  8.0953e-01, -1.2493e+00, -2.5607e+00, -6.3088e-01,
         -1.8721e-01, -7.4575e-01,  1.0072e+00,  1.3309e+00,  7.0461e-01,
         -1.7939e-01,  1.1260e+00, -8.7020e-01,  2.5821e+00,  2.5286e+00,
          2.0509e+00,  3.3878e+00,  2.7445e+00,  3.8563e+00,  2.8725e+00,
          2.9410e+00,  8.5213e-01,  3.7949e+00,  3.1803e+00,  4.1433e+00,
          2.2746e-01,  2.0610e+00,  1.5298e+00,  4.7920e-01,  2.2985e+00,
          1.3052e+00, -1.1126e+00,  3.8941e-01,  2.2656e+00,  3.5423e+00,
          4.7905e+00,  3.3872e+00,  5.9888e+00,  3.2111e+00,  2.1360e+00,
          4.5508e-01,  5.5269e+00,  6.

---

## 3. Create a quantization simulation model and determine quantized accuracy

### Fold Batch Norm layers

Before calculating the simulated quantized accuracy using QuantizationSimModel, fold the BatchNorm (BN) layers into adjacent Convolutional layers. The BN layers that cannot be folded are left as they are.

BN folding improves inference performance on quantized runtimes but can degrade accuracy on these platforms. This step simulates this on-target drop in accuracy. 

**3.1 Use the following code to call AIMET to fold the BN layers in-place on the given model.**

In [16]:
from aimet_torch.batch_norm_fold import fold_all_batch_norms

_ = fold_all_batch_norms(model, input_shapes=(1, 3, 224, 224))

[2024-12-20 10:45:37,981] [WARNING] [real_accelerator.py:162:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2024-12-20 10:45:37,982] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cpu (auto detect)


df: /root/.triton/autotune: No such file or directory


In [17]:
len(_)

20

### Create the Quantization Sim Model

**3.2 Use AIMET to create a QuantizationSimModel.**

 In this step, AIMET inserts fake quantization ops in the model graph and configures them.

Key parameters:

- Setting **default_output_bw** to 8 performs all activation quantizations in the model using integer 8-bit precision
- Setting **default_param_bw** to 8 performs all parameter quantizations in the model using integer 8-bit precision
- **num_batches** is the number of batches to use to compute encodings. Only five batches are used here for the sake of speed

See [QuantizationSimModel in the AIMET API documentation](https://quic.github.io/aimet-pages/AimetDocs/api_docs/torch_quantsim.html#aimet_torch.quantsim.QuantizationSimModel.compute_encodings) for a full explanation of the parameters.

In [18]:
from aimet_common.defs import QuantScheme
from aimet_torch.v1.quantsim import QuantizationSimModel

dummy_input = torch.rand(1, 3, 224, 224)    # Shape for each ImageNet sample is (3 channels) x (224 height) x (224 width)
if use_cuda:
    dummy_input = dummy_input.cuda()

sim = QuantizationSimModel(model=model,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           dummy_input=dummy_input,
                           default_output_bw=8,
                           default_param_bw=8,
                           config_file="/home/shayan/Desktop/aimet/my_config.json")

2024-12-20 10:46:01,360 - Quant - INFO - Unsupported op type Squeeze
2024-12-20 10:46:01,361 - Quant - INFO - Unsupported op type Mean
2024-12-20 10:46:01,362 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


---
**3.3 Print the model to verify the modifications AIMET has made. **

Note that AIMET has added quantization wrapper layers. 

<div class="alert alert-info">

Note

Use sim.model to access the modified PyTorch model. By default, AIMET creates a copy of the original model prior to modifying it. There is a parameter to override this behavior.

</div>

In [11]:
print(sim.model)

GraphModule(
  (conv1): StaticGridQuantWrapper(
    (_module_to_wrap): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  )
  (bn1): Identity()
  (relu): StaticGridQuantWrapper(
    (_module_to_wrap): ReLU(inplace=True)
  )
  (maxpool): StaticGridQuantWrapper(
    (_module_to_wrap): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Module(
    (0): Module(
      (conv1): StaticGridQuantWrapper(
        (_module_to_wrap): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (bn1): Identity()
      (relu): StaticGridQuantWrapper(
        (_module_to_wrap): ReLU(inplace=True)
      )
      (conv2): StaticGridQuantWrapper(
        (_module_to_wrap): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (bn2): Identity()
      (module_add): StaticGridQuantWrapper(
        (_module_to_wrap): Add()
      )
      (module_relu_1): StaticGridQuantWrapper(
        (_module_to_wrap): ReLU(

---
Note also that AIMET has configured the added fake quantization nodes, which AIMET refers to as "quantizers". 

**3.4 Print the sim object to see the quantizers.**

In [12]:
sim.__dict__()

{'conv1': {'inputs': {'0': {}},
  'params': {'weight': {}, 'bias': {}},
  'outputs': {'0': {}}},
 'relu': {'inputs': {'0': {}}, 'params': {}, 'outputs': {'0': {}}},
 'maxpool': {'inputs': {'0': {}}, 'params': {}, 'outputs': {'0': {}}},
 'layer1.0.conv1': {'inputs': {'0': {}},
  'params': {'weight': {}, 'bias': {}},
  'outputs': {'0': {}}},
 'layer1.0.relu': {'inputs': {'0': {}}, 'params': {}, 'outputs': {'0': {}}},
 'layer1.0.conv2': {'inputs': {'0': {}},
  'params': {'weight': {}, 'bias': {}},
  'outputs': {'0': {}}},
 'layer1.0.module_add': {'inputs': {'0': {}, '1': {}},
  'params': {},
  'outputs': {'0': {}}},
 'layer1.0.module_relu_1': {'inputs': {'0': {}},
  'params': {},
  'outputs': {'0': {}}},
 'layer1.1.conv1': {'inputs': {'0': {}},
  'params': {'weight': {}, 'bias': {}},
  'outputs': {'0': {}}},
 'layer1.1.relu': {'inputs': {'0': {}}, 'params': {}, 'outputs': {'0': {}}},
 'layer1.1.conv2': {'inputs': {'0': {}},
  'params': {'weight': {}, 'bias': {}},
  'outputs': {'0': {}}},


---
AIMET has added quantizer nodes to the model graph, but before the sim model can be used for inference or training, scale and offset quantization parameters must be calculated for each quantizer node by passing unlabeled data samples through the model to collect range statistics. This process is sometimes referred to as calibration. AIMET refers to it as "computing encodings".

**3.5 Create a routine to pass unlabeled data samples through the model.** 

The following code is one way to write a routine that passes unlabeled samples through the model to compute encodings. It uses the existing train or validation data loader to extract samples and pass them to the model. Since there is no need to compute loss metrics, it ignores the model output.  

In [19]:
data_loader = ImageNetDataPipeline.get_val_dataloader()
data_loader.batch_size


2024-12-20 10:46:08,713 - Dataloader - INFO - Dataset consists of 3 images in 1 classes


1

In [20]:

def pass_calibration_data(sim_model, use_cuda):
    data_loader = ImageNetDataPipeline.get_val_dataloader()
    batch_size = data_loader.batch_size

    if use_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    sim_model.eval()
    samples = 1000

    batch_cntr = 0
    idx = 0
    with torch.no_grad():
        for path, input_data, target_data in data_loader:
            # if "cf135f199d8c7a9d0dce9aa35acfb4c70c14e0aa" not in path[0]:
            #     continue
            # if "cf" in path[0]:
            print(path)
            inputs_batch = input_data.to(device)
            sim_model(inputs_batch)
            break


A few notes regarding the data samples:

- A very small percentage of the data samples are needed. For example, the training dataset for ImageNet has 1M samples; 500 or 1000 suffice to compute encodings.
- The samples should be reasonably well distributed. While it's not necessary to cover all classes, avoid extreme scenarios like using only dark or only light samples. That is, using only pictures captured at night, say, could skew the results.

---

**3.6 Call AIMET to use the routine to pass data through the model and compute the quantization encodings.** 

Encodings here refer to scale and offset quantization parameters.

In [21]:
sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

2024-12-20 10:46:16,564 - Dataloader - INFO - Dataset consists of 3 images in 1 classes
('/home/shayan/Desktop/aimet/Examples/torch/quantization/val/9/cf135f199d8c7a9d0dce9aa35acfb4c70c14e0aa.jpeg',)


In [17]:
print(sim)

-------------------------
Quantized Model Report
-------------------------
----------------------------------------------------------
Layer: conv1
  Input[0]: bw=8, encoding-present=True
    StaticGrid TensorQuantizer:
    quant-scheme:QuantScheme.post_training_tf, round_mode=RoundingMode.ROUND_NEAREST, bitwidth=8, enabled=True
    min:-2.231783390045166, max=2.231783390045166, delta=0.017573097559410757, offset=-127.0
  -------
  Param[weight]: bw=8, encoding-present=True
    StaticGrid TensorQuantizer:
    quant-scheme:QuantScheme.post_training_tf, round_mode=RoundingMode.ROUND_NEAREST, bitwidth=8, enabled=True
    min:-0.39387544989585876, max=0.39387544989585876, delta=0.0031013814952429823, offset=-127.0
  -------
  Param[bias]: bw=8, encoding-present=True
    StaticGrid TensorQuantizer:
    quant-scheme:QuantScheme.post_training_tf, round_mode=RoundingMode.ROUND_NEAREST, bitwidth=8, enabled=True
    min:-0.6932891607284546, max=0.6932891607284546, delta=0.005458969769515391, offs

In [17]:
import json
# with open("/home/shayan/Desktop/aimet/quantsim_report/tf/cf135f199d8c7a9d0dce9aa35acfb4c70c14e0aa.json", "w") as f:
# with open("/home/shayan/Desktop/aimet/quantsim_report/tf/c31381770ecd2ebd7530960f11a9c6ca56fd7f62.json", "w") as f:
with open("/home/shayan/Desktop/aimet/quantsim_report/tf/c3913df91d5dea761373473b38c96f3a1c9711b6.json", "w") as f:
    json.dump(sim.__dict__(), f, indent=4)

---

The QuantizationSim model is now ready to be used for inference or training. 

**3.7 Pass the model to the same evaluation routine as before to calculate a simulated quantized accuracy score for INT8 quantization for comparison with the FP32 score.**

In [23]:
quant_outputs = forward(sim.model, ImageNetDataPipeline.get_val_dataloader())
quant_outputs.shape

2024-12-20 10:47:23,613 - Dataloader - INFO - Dataset consists of 3 images in 1 classes
('/home/shayan/Desktop/aimet/Examples/torch/quantization/val/9/cf135f199d8c7a9d0dce9aa35acfb4c70c14e0aa.jpeg',)


torch.Size([1, 1000])

In [30]:
import numpy as np

torch.mean((fp32_output - quant_outputs)**2)

tensor(0.0698)

In [18]:
accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(accuracy)

2024-12-17 07:48:46,724 - Dataloader - INFO - Dataset consists of 1110 images in 111 classes
2024-12-17 07:48:46,726 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-12-17 07:48:46,727 - Eval - INFO - Evaluating nn.Module for 1110 iterations with batch_size 1


  0%|          | 0/1110 [00:00<?, ?it/s]

100%|██████████| 1110/1110 [01:31<00:00, 12.16it/s]

2024-12-17 07:50:18,060 - Eval - INFO - Avg accuracy Top 1: 0.000000 Avg accuracy Top 5: 0.450450 on validation Dataset
0.0


---
## 4. Apply CLE and BC

### CLE

**4.1 Perform CLE**

The next cell performs cross-layer equalization on the model. As noted before, the function folds batch norms, applies cross-layer scaling, and then folds high biases.

<div class="alert alert-info">

Note

The CLE procedure needs BN statistics. If a BN folded model is provided, CLE runs the cross-layer scaling (CLS) optimization step but skips the high-bias absorption (HBA) step. To avoid this, load the original model again before running CLE.

</div>

<div class="alert alert-info">

Note

CLE equalizes the model in-place.

</div>



In [ ]:
model = resnet18(pretrained=True) # Reload the model
model = prepare_model(model)

use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    model.to(torch.device('cuda'))

In [ ]:
from aimet_torch.cross_layer_equalization import equalize_model

equalize_model(model, input_shapes=(1, 3, 224, 224))

---

**4.2 Compute the accuracy of the equalized model.** 

Create a simulation model as before and evaluate it to determine simulated quantized accuracy.

In [ ]:
sim = QuantizationSimModel(model=model,
                           quant_scheme=QuantScheme.post_training_tf,
                           dummy_input=dummy_input,
                           default_output_bw=8,
                           default_param_bw=8)

sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(accuracy)


### Bias Correction

**4.3 Apply AIMET Bias Correction to the equalized model from the previous step. **

Bias correction uses a reference FP32 model and a QuantizationSimModel to perform the procedure. For details see [Post-training quantization techiques](https://quic.github.io/aimet-pages/releases/latest/user_guide/post_training_quant_techniques.html#) in the [AIMET User Guide](https://quic.github.io/aimet-pages/releases/latest/user_guide/index.html).

Key parameters:

- **num_quant_samples** is the number of samples used for computing encodings. Set low in this example to save time. A more typical number is 500-1000.
- **num_bias_correct_samples** is the number of samples used for bias correction. Set low in this example to save time. A more typical number is 1000-2000.
- **data_loader**: BC uses unlabeled data samples from this data loader.

In [ ]:
from aimet_torch.v1.quantsim import QuantParams
from aimet_torch.bias_correction import correct_bias

data_loader = ImageNetDataPipeline.get_val_dataloader()

bc_params = QuantParams(weight_bw=8, act_bw=8, round_mode="nearest",
                        quant_scheme=QuantScheme.post_training_tf_enhanced)

correct_bias(model, bc_params, num_quant_samples=16,
             data_loader=data_loader, num_bias_correct_samples=16)

---

The QuantizationSim model is now ready to be used for inference or training. 

**4.4 Again calculate a simulated quantized accuracy, this time for the bias-corrected model.**

In [ ]:
sim = QuantizationSimModel(model=model,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           dummy_input=dummy_input,
                           default_output_bw=8,
                           default_param_bw=8)

sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(accuracy)

---

There might be little gain in accuracy after this limited application of CLE and BC. Experiment with the hyper-parameters to get better results.

## Next steps

The next step is to export this model for installation on the target.

**Export the model and encodings.**

- Export the model with the updated weights but without the fake quant ops. 
- Export the encodings (scale and offset quantization parameters). AIMET QuantizationSimModel provides an export API for this purpose.

The following code performs these exports.

In [ ]:
os.makedirs('./output/', exist_ok=True)
dummy_input = dummy_input.cpu()
sim.export(path='./output/', filename_prefix='resnet18_after_cle_bc', dummy_input=dummy_input)

## For more information

See the [AIMET API docs](https://quic.github.io/aimet-pages/AimetDocs/api_docs/index.html) for details about the AIMET APIs and optional parameters.

See the [other example notebooks](https://github.com/quic/aimet/tree/develop/Examples/torch/quantization) to learn how to use other AIMET post-training quantization techniques.

To learn more about these techniques, see ["Data-Free Quantization Through Weight Equalization and Bias Correction"](https://arxiv.org/abs/1906.04721) from ICCV 2019.